<a href="https://colab.research.google.com/github/ArkS0001/Tacotron2/blob/main/AudFAKE_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Install necessary packages
!pip install wget torch numpy scipy soundfile librosa pyaudio webrtcvad

# Clone the VITS repository for vocoder usage
!git clone https://github.com/jaywalnut310/vits.git
!pip install -r vits/requirements.txt
!apt-get install -y espeak


  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... canceled
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 92, in resolve
    result = self._result = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/re

In [8]:
# Install PortAudio library
!apt-get install -y portaudio19-dev

# Install PyAudio
!pip install pyaudio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudiocpp0 portaudio19-dev
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 122 kB of archives.
After this operation, 703 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 122 kB in 2s (75.1 kB/s)
Selecting previously unselected package libportaudiocpp0:amd64.
(Reading database ... 122243 files and directories currently installed.)
Preparing to unpack .../libportaudiocpp0_19.6.0-1.1_amd64.deb ...
Unpacking libportaudiocpp0:amd64 (19.6.0-1.1) ...
Selecting previously unselected package portaudio19-dev:

In [10]:
import wget
import tarfile
import os

# Download and extract the LJSpeech dataset
#url = 'https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2'
#wget.download(url)

#with tarfile.open('LJSpeech-1.1.tar.bz2', 'r:bz2') as tar:
#    tar.extractall()

# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import librosa
import soundfile as sf
from torch.utils.data import Dataset, DataLoader
from vits.text import text_to_sequence
from vits.text.symbols import symbols

class LJSpeechDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.metadata = self._load_metadata()

    def _load_metadata(self):
        metadata_path = os.path.join(self.data_dir, 'metadata.csv')
        with open(metadata_path, 'r') as f:
            metadata = [line.strip().split('|') for line in f]
        return metadata

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        wav_path = os.path.join(self.data_dir, 'wavs', self.metadata[idx][0] + '.wav')
        y, sr = sf.read(wav_path)
        y = librosa.resample(y, sr, 22050).astype(np.float32)
        text = self.metadata[idx][1]
        mel_spectrogram = librosa.feature.melspectrogram(y, sr=22050, n_fft=1024, hop_length=256, n_mels=80)
        mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

        return torch.tensor(text_to_sequence(text, ['english_cleaners']), dtype=torch.long), torch.tensor(mel_spectrogram, dtype=torch.float32)

# Create the dataset and data loader
dataset = LJSpeechDataset(data_dir='LJSpeech-1.1')
data_loader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))


ModuleNotFoundError: No module named 'text'

In [12]:
!git clone https://github.com/jaywalnut310/vits.git
%cd vits
!pip install -r requirements.txt


fatal: destination path 'vits' already exists and is not an empty directory.
/content/vits
  Using cached Cython-0.29.21-py2.py3-none-any.whl (974 kB)
  Using cached librosa-0.8.0.tar.gz (183 kB)
  Preparing metadata (setup.py) ... done
  Using cached matplotlib-3.3.1.tar.gz (38.8 MB)
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.18.5.zip (5.4 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Seq2Seq, self).__init__()
        self.encoder = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.decoder = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        _, (hidden, cell) = self.encoder(x)
        outputs, _ = self.decoder(x, (hidden, cell))
        outputs = self.fc(outputs)
        return outputs

# Hyperparameters
input_dim = 128  # Size of text embeddings
hidden_dim = 256
output_dim = 80  # Number of mel-spectrogram bins

# Instantiate the model
model = Seq2Seq(input_dim, hidden_dim, output_dim).cuda()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
num_epochs = 50

for epoch in range(num_epochs):
    for texts, mels in data_loader:
        texts = [torch.tensor(t, dtype=torch.long) for t in texts]
        mels = [torch.tensor(m, dtype=torch.float32) for m in mels]

        texts = torch.nn.utils.rnn.pad_sequence(texts, batch_first=True).cuda()
        mels = torch.nn.utils.rnn.pad_sequence(mels, batch_first=True).cuda()

        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, mels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


In [ ]:
import torch
from waveglow.denoiser import Denoiser

# Load pre-trained WaveGlow model
waveglow = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_waveglow')
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow.eval().cuda()
denoiser = Denoiser(waveglow).cuda()

def mel_to_audio(mel):
    mel = torch.tensor(mel).unsqueeze(0).cuda()
    with torch.no_grad():
        audio = waveglow.infer(mel, sigma=0.666)
        audio = denoiser(audio, strength=0.01)[:, 0]
    return audio.cpu().numpy()

# Example usage
text = "Hello, how are you?"
text_seq = torch.tensor(text_to_sequence(text, ['english_cleaners']), dtype=torch.long).unsqueeze(0).cuda()
with torch.no_grad():
    mel = model(text_seq).cpu().numpy()
audio = mel_to_audio(mel)

# Save the synthesized audio to a file
sf.write('synthesized_output.wav', audio, 22050)

# Playback the audio
import IPython.display as ipd
ipd.Audio('synthesized_output.wav')
